<h2 align=center> Toxic Comments Classification using 1D CNN with Keras</h2>

### Task 1: Import Packages and Functions

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.2.0


### Task 2: Load and Explore Data

In [2]:
# Load data
train_df = pd.read_csv("train.csv").fillna(" ")
train_df.sample(10, random_state = 1)

,comment_text,toxic
24915,"YOU ARE A FAT, GEEKY PRICK WHO HAS NOTHING TO ...",1
75819,Agent X2: Basically thanks - with a 'little' m...,0
53891,Why are my posts being deleted? \n\nI have tri...,0
154159,"""\n\n Controlled Demolitions and Common Sense ...",0
13040,I do not understand your reply. //Blaxthos ( ...,0
123190,Is this the bizarro world? Removing content is...,0
33626,"Well, WP:RS says that articles should use reli...",0
1150,Oh hear me go someone removes all my pages i g...,0
48633,can't believe this article was deleted\nI'm su...,0
42817,"""\n\n Comments on GamerGate Workshop page \n\n...",0


In [3]:
x = train_df['comment_text'].values
print(x)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

In [4]:
# View few toxic comments
train_df.loc[train_df['toxic'] == 1].sample(10, random_state = 10)

,comment_text,toxic
30575,"""\n Are you asking if we would ever block some...",1
36038,Amen my sock-pupinski brother!! Hack-a-long: p...,1
19730,JUST READ THE FUCKING CENSUS DATA. Do you thin...,1
72261,u no wat ass hole go fuck your self and stop b...,1
77923,Go fuck yourself \n\nEat shit and die. 68.0.11...,1
16382,A big thank you\n\nYou have just blocked me in...,1
41247,A message from Jasonceyre \n\n f uc k you ho...,1
69311,"I'm gonna get the ACLU on you, you racist anti...",1
142032,"Hey Freepsbane, does your mother know that you...",1
5613,You guys suk. You really really suck. Fuck you...,1


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
comments = train_df['comment_text'].loc[train_df['toxic']==1].values
wordcloud = WordCloud(
    width = 640,
    height = 640,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(comments))
fig = plt.figure(
    figsize = (12, 8),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [5]:
y = train_df['toxic'].values
print(y)

[0 0 0 ... 0 0 0]


In [6]:
# Plot frequency of toxic comments
import matplotlib.pyplot as plt
train_df['toxic'].plot(kind = 'hist', title = 'Distribution of Toxic Comments')
plt.show()

<Figure size 640x480 with 1 Axes>

In [7]:
train_df['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

### Task 3: Data Prep — Tokenize and Pad Text Data

In [8]:
max_features = 20000
max_text_length = 400

In [9]:
x_tokenizer = text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
x_tokenizer.word_counts

OrderedDict([('explanation', 1771),
             ('why', 17818),
             ('the', 496540),
             ('edits', 9957),
             ('made', 9678),
             ('under', 7001),
             ('my', 45619),
             ('username', 1823),
             ('hardcore', 166),
             ('metallica', 35),
             ('fan', 1025),
             ('were', 15421),
             ('reverted', 3902),
             ('they', 27318),
             ("weren't", 424),
             ('vandalisms', 38),
             ('just', 27806),
             ('closure', 88),
             ('on', 89968),
             ('some', 22586),
             ('gas', 331),
             ('after', 8490),
             ('i', 204397),
             ('voted', 302),
             ('at', 39545),
             ('new', 10665),
             ('york', 912),
             ('dolls', 24),
             ('fac', 321),
             ('and', 224192),
             ('please', 29780),
             ("don't", 22917),
             ('remove', 5140),
          

In [10]:
x_tokenized = x_tokenizer.texts_to_sequences(x)
x_train_val = sequence.pad_sequences(x_tokenized, maxlen = max_text_length)

In [11]:
x_train_val

array([[    0,     0,     0, ...,  4583,  2273,   985],
       [    0,     0,     0, ...,   589,  8377,   182],
       [    0,     0,     0, ...,     1,   737,   468],
       ...,
       [    0,     0,     0, ...,  3509, 13675,  4528],
       [    0,     0,     0, ...,   151,    34,    11],
       [    0,     0,     0, ...,  1627,  2056,    88]])

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import text, sequence
tk = Tokenizer()
texts = ["my name is far", "my name is","your name is"]
tk.fit_on_texts(texts)
print(tk.word_counts)
print(tk.index_word)
tk.num_words = 2
print(tk.texts_to_sequences(texts))
tk.num_words = 3
tk.texts_to_sequences(texts)
tk.num_words = 4 
tk.texts_to_sequences(texts)
tk.num_words = 5
tk.texts_to_sequences(texts)
sequence.pad_sequences(tk.texts_to_sequences(texts), maxlen=10)

### Task 4: Prepare Embedding Matrix with Pre-trained GloVe Embeddings

In [12]:
#Download GloVe from -> http://nlp.stanford.edu/data/glove.6B.zip

embedding_dim = 100
embeddings_index = dict()
f = open('glove.6B.100d.txt','r', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()
print(f'Found {len(embeddings_index)} word vectors.')

Found 400000 word vectors.


In [13]:
x_tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'have': 18,
 'are': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'page': 27,
 'wikipedia': 28,
 'my': 29,
 'an': 30,
 'from': 31,
 'by': 32,
 'do': 33,
 'at': 34,
 'me': 35,
 'about': 36,
 'so': 37,
 'talk': 38,
 'what': 39,
 'can': 40,
 'there': 41,
 'all': 42,
 'has': 43,
 'will': 44,
 'please': 45,
 'no': 46,
 'would': 47,
 'one': 48,
 'like': 49,
 'just': 50,
 'they': 51,
 'he': 52,
 'which': 53,
 'any': 54,
 'been': 55,
 'should': 56,
 'more': 57,
 'we': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'here': 63,
 'see': 64,
 'also': 65,
 'his': 66,
 'think': 67,
 'because': 68,
 'know': 69,
 'how': 70,
 'edit': 71,
 'am': 72,
 "i'm": 73,
 'people': 74,
 'why': 75,
 'up': 76,
 'only': 77,
 "it's": 78,
 'out': 79,
 'articles': 80,
 'use': 81,

In [14]:
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, index in x_tokenizer.word_index.items():
    if index > max_features - 1:
        break
    else:
        embeddings_vector = embeddings_index.get(word)
        if embeddings_vector is not None:
            embedding_matrix[index] = embeddings_vector

### Task 5: Create the Embedding Layer

In [15]:
#Using pretrained embedding matrix from previous lines
model = Sequential()
model.add(Embedding(max_features,
                    embedding_dim,
                    embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
                    trainable = False))
model.add(Dropout(0.2))

### Task 6: Build the Model

In [16]:
filters = 250
kernel_size = 3
hidden_dims = 250

In [17]:
model.add(Conv1D(filters,
                 kernel_size,
                 padding = 'valid'))
model.add(MaxPooling1D())
model.add(Conv1D(filters,
                 5,
                 padding = 'valid',
                 activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2000000   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         75250     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 250)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 250)         312750    
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               6

In [18]:
#We use binary cross entropy as we have only 2 categories
#For multiple categories, use categorical cross entropy
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

### Task 7: Train Model

In [19]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y, test_size = 0.15, random_state = 1)

In [20]:
%%time
batch_size = 32
epochs = 3

model.fit(x_train, y_train, batch_size = batch_size, epochs = 3, validation_data = (x_val, y_val))

Epoch 1/3
4239/4239 [==============================] - 900s 212ms/step - loss: 0.1536 - accuracy: 0.9443 - val_loss: 0.1282 - val_accuracy: 0.9525
Epoch 2/3
4239/4239 [==============================] - 906s 214ms/step - loss: 0.1329 - accuracy: 0.9507 - val_loss: 0.1351 - val_accuracy: 0.9469
Epoch 3/3
4239/4239 [==============================] - 920s 217ms/step - loss: 0.1239 - accuracy: 0.9537 - val_loss: 0.1211 - val_accuracy: 0.9538
Wall time: 45min 31s


### Task 8: Evaluate Model

In [21]:
test_df = pd.read_csv("test.csv")

In [22]:
x_test = test_df['comment_text'].values

In [25]:
x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)
x_testing = sequence.pad_sequences(x_test_tokenized, maxlen = max_text_length)

In [26]:
y_testing = model.predict(x_testing, verbose = 1, batch_size = 32)

4787/4787 [==============================] - 227s 47ms/step


In [27]:
y_testing.shape

(153164, 1)

In [28]:
y_testing[0]

array([0.94285166], dtype=float32)

In [29]:
test_df['Toxic'] = ['not toxic' if x < 0.5 else 'toxic' for x in y_testing]
test_df[['comment_text', 'Toxic']].head(20) 

,comment_text,Toxic
0,Yo bitch Ja Rule is more succesful then you'll...,toxic
1,== From RfC == \n\n The title is fine as it is...,not toxic
2,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",not toxic
3,":If you have a look back at the source, the in...",not toxic
4,I don't anonymously edit articles at all.,not toxic
5,Thank you for understanding. I think very high...,not toxic
6,Please do not add nonsense to Wikipedia. Such ...,not toxic
7,:Dear god this site is horrible.,toxic
8,""" \n Only a fool can believe in such numbers. ...",toxic
9,== Double Redirects == \n\n When fixing double...,not toxic


In [34]:
test_df[['comment_text', 'Toxic']].sample(20, random_state = 10)

,comment_text,Toxic
92177,""" \n {| style=""""background-color:#ddcef2; padd...",not toxic
109153,== Editor Boredwhytekid need to learn how to l...,not toxic
134240,** I cleaned up a lot of this Sock Puppet's un...,toxic
143218,== FUCKKKKKKK Philidelphia GO RAYS== \n\n \n ...,toxic
2727,you're a little cunt,toxic
21269,==Sockpuppet== \n I am not a sockpuppet,not toxic
25055,**Your obviously a 12 year old..,not toxic
54811,Shadowjams? What a hell of a lame name lol. Di...,toxic
131889,""" \n :There's a thread at ANI which gives more...",not toxic
148283,this is my fucking page i can do wat i want ow...,toxic
